In [2]:
from adversarial.defences.adv_classifier import *
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tqdm import tqdm

In [3]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [4]:
import numpy as np

In [5]:
# dataloader
from adversarial.utils import dataloader
from adversarial.attacks import random_attack

In [6]:
BATCH_SIZE = 16
EPSILONS = np.arange(0.01, 0.5, step=0.02)
ETA = 1e-5
EPOCHS = 6

In [7]:
train_data_loader = dataloader.DataLoader('./data', 16, training=True)
test_data_loader = dataloader.DataLoader('./data', 16, training= False)

In [8]:
from adversarial.defences import EnsembleModel
from adversarial.models import *

random_attack(앙상블 모델 넣을건데, x_image, adversarial_label, epsilon은 배열로)  
epsilon = np.arange(0.01,0.5,0.02)  
random_attack은 adversarial image를 리턴해준다. 

In [9]:
target_model= AdvClassifier()

In [10]:
# train

criterion = losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = ETA)

train_losses = tf.keras.metrics.CategoricalCrossentropy()
train_metrics = tf.keras.metrics.CategoricalAccuracy()

test_losses = tf.keras.metrics.CategoricalCrossentropy()
test_metrics = tf.keras.metrics.CategoricalAccuracy()

In [11]:
ensemble_model = EnsembleModel([
    VGG19, VGG16, MobileNet, MobileNetV2, ResNet50, ResNet50V2
], devices = [0])

In [12]:
labels = [[0,1] ,[1,0]]
    
y_batch = []
    
    
for label in labels : 
    for time in range(16) :
        y_batch.append(label)

y_tensor = tf.convert_to_tensor(y_batch)

In [22]:
def train_step(images, adv_labels):

    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    with tf.GradientTape() as tape:
        outputs = target_model(x_batch, training = True)
      #  print('y_batch : ' ,y_batch)
       # print('outputs : ' ,outputs)
        loss = criterion(y_batch, outputs)
    train_losses.update_state(y_batch, outputs)
    train_metrics.update_state(y_batch, outputs)
        
    grads = tape.gradient(loss, target_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, target_model.trainable_variables))

In [14]:
def test_step(images, labels, adv_labels) :
    adv,_, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    x_batch = tf.concat([images, adv], axis=0)
    y_batch = y_tensor
    #y_batch = tf.concat([tf.zeros(16, dtype = tf.int32),
     #                    tf.ones(16, dtype = tf.int32)], 
      #                   axis = 0)
    
    predictions = target_model(x_batch, training= False)
    loss = tf.keras.losses.binary_crossentropy(y_batch, predictions)
    test_losses.update_state(y_batch, predictions)
    test_metrics.update_state(y_batch, predictions)

In [18]:
def train():

    train_loss_list = []
    test_loss_list = []

    for e in range(EPOCHS):
        for x_batch, y_batch, adv_batch in tqdm(iter(train_data_loader)):
            train_step(x_batch, adv_batch)

        for x_batch, y_batch, adv_batch in tqdm(iter(test_data_loader)):
            test_step(x_batch, y_batch, adv_batch)


        train_loss = train_losses.result()
        train_acc = train_metrics.result()

        test_loss = test_losses.result()
        test_acc = test_metrics.result()

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_losses.reset_states()
        train_metrics.reset_states()
        test_losses.reset_states()
        test_metrics.reset_states()
        
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [ ]:
def load_custom_weights(target, path):
    with open(path, "rb") as f:
        data = pickle.load(f)
        target.set_weights(data)

def save_custom_weights(target, path):
    with open(path, "wb") as f:
        pickle.dump(target.get_weights(), f)

In [ ]:
target_model.summary()

In [ ]:
train()

806it [33:55,  2.53s/it]
32it [00:39,  1.22s/it]
0it [00:00, ?it/s]

Epochs 1/6, train loss: 2.03927422, train acc: 0.6486, test loss: 1.93918061, test acc: 0.6123


806it [32:17,  2.40s/it]
32it [01:20,  2.52s/it]
0it [00:00, ?it/s]

Epochs 2/6, train loss: 1.98466408, train acc: 0.6549, test loss: 2.05114889, test acc: 0.6934


98it [05:04,  2.45s/it]

In [ ]:
save_custom_weights(target_model, './adv_classifier_6.3.pkl')